In [1]:
import pandas as pd
import numpy as np
import os
import re
import glob

from tqdm.notebook import tqdm

np.random.seed(10)

# Считывание всех данных

In [2]:
def ReadToDf(folder: str):
    df = pd.DataFrame(columns=['univ', 'prog', 'name', 'text'])
    for file in tqdm(glob.glob('{}/*/*/*.txt'.format(folder))):
        splitted = file.split('\\')
        name = splitted[-1][:-4]
        u = splitted[-3]
        op = splitted[-2]
        
        try:
            text = open(file, 'r').read()
        except Exception as e:
            print(e)
            print(file)
            text = open(file, 'r', encoding='utf-8').read()
        
        df = df.append({'univ':u, 'prog':op, 'name':name, 'text':name + ' ' + text.lower()}, ignore_index=True)
    return df

In [3]:
df = ReadToDf('files')

  0%|          | 0/3022 [00:00<?, ?it/s]

# Лемматизация текста
* https://habr.com/ru/post/205360/
* https://russianblogs.com/article/9814548532/

In [4]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

a = ord('а')
default_stop = set(stopwords.words('russian') + [chr(i) for i in range(a, a + 32)] + [_ for _ in range(10)])

Лемматизацию текста можно проводить с помощью двух популярных библиотек для русского языка: `pymorphy2` и `pymystem3`. Выбор для данной работы -- `pymystem3`. Основное преимущество: простота использования и контекстная лемматизация.

In [5]:
from pymystem3 import Mystem
ms = Mystem()

regs = [
    r'аннотация', r'\sический\s', r'\sопоп\s',
    r'\sшт\s', r'\sдр\s', r'\sго\s', r'\sита\s', r'\sия\s', r'\sв?ах\s',
    r'\sсей\s', r'\sтий\s', r'\sвать\s', r'\sдея\s',
    r'\ d?\d*?\ ', '0302',
    r'[·•—“”„№¶…>%=’]',
]

def lemmatize(text):
    lemmas = ms.lemmatize(text)
    text = ''.join(lemmas).rstrip('\n')
    for reg in regs:
        text = re.sub(reg, '', text)
    return text

Предыдущая работы была проделана с целью очистки текста так, чтобы фильная обработка включала в себя только удаление лишних слов. Таким образом, были удалены лишние конструкции, такие как, например, "20 часов", "целями дисциплины являются", "основные компетенции" и т. д., удалены знаки пунктуации, кроме дефиса: некоторые важные составные слова могут существенно повлиять на качество модели (SQL-запрос, web-разработка, т. д.).

Для начала удалим все стоп-слова: союзы, предлоги и т. д., затем проведем количественный анализ слов, и из каждого текста удалим определенный процент часто- и редковстречаемых слов.

Решено было удалить и кодировки компетенций, которые изначально планировалось оставить. Такое решение позволило в три раза уменьшить словарь.

In [6]:
import time

comp_regs = [
    r'ОПК[\-\ ]?\d\d?',
    r'ПК[\-\ ]?\d\d?',
    r'УК[\-\ ]?\d\d?',
    r'ОК[\-\ ]?\d\d?',
    r'ИДЫ[\-\ ]?\d\d?',
    r'ИДК[\-\ ]?\d\d?',
]

def process_text(text, f_lemmatizer=lemmatize, wstop=default_stop, counter=None):
    if f_lemmatizer is not None:
        text = f_lemmatizer(text)
    
    for reg in comp_regs:
        text = re.sub(reg, '', text, flags=re.IGNORECASE)
              
    text = [word for word in word_tokenize(text) if word not in wstop]
    if counter is not None:
        counter.update(text)
    return ' '.join(text)

fd_counter = FreqDist()

start = time.time()
df.text = df.text.apply(process_text, counter=fd_counter)
print('Time spent: {} s.'.format(time.time() - start))

Time spent: 2882.502287387848 s.


In [7]:
len(fd_counter)

19027

Код выше выполняется около часа, поэтому перепишем результат его работы в json.

In [8]:
import json
jsonStr = json.dumps(dict(fd_counter.most_common()), ensure_ascii=False)
file = open('JsonCounter.json', 'w')
file.write(jsonStr)
file.close()

Переписывание файлов.

In [9]:
def rewrite(folder: str, df):
    for row in df.iterrows():
        r = row[1]
        path = folder + '\\' + r['univ'] + '\\' + r['prog']
        try:
            os.makedirs(path)
        except FileExistsError:
            pass
        
        filename = path + '\\' + r['name'] + '.txt'

        file_ = open(filename, 'w', encoding='ansi')
        try:
            file_.write(r['text'])
        except Exception as e:
            print('FATAL:', e)
            print(row[0], filename)
            file_.close() 
            break
        file_.close()

In [10]:
# rewrite('files_lemm', df)

Если посмотреть на получившийся словарь, то среди редковстречаемых слов можно обнаружить множество выбросов: отрывки слов, необычные символы, и т. д. Проблема заключается в том, что такие слова часто встречаются столько же раз, сколько и семантически важные слова, что может существенно испортить качество кластеризации. Данная проблема возникла из-за несовершенства парсинга, а так же опечаток, допущенных в процессе составления документа авторами.

Чтобы сгладить углы проблемы, удалим те слова, которые начинаются не с буквы: если лексикографически отсортировать получившийся словарь, то легко понять, что первые слова, стоящие до слов, начинающийхся на английскую `a` являются невалидными. Примеры таких слов: 19гfreeware, 25002900лексическихединиц, 14экзаменаци.

Конечно, это не панацея, и опечатки все равно останутся, однако их будет меньше.

In [11]:
corrected_dict = {}
typo_words = []
for item in sorted(fd_counter.items()):
    if not str.isalpha(item[0][0]):
        typo_words.append(item[0])
        continue
    corrected_dict[item[0]] = item[1]
len(corrected_dict)

18723

In [12]:
d = FreqDist(corrected_dict)
d

FreqDist({'система': 10631, 'метод': 8273, 'задача': 7434, 'навык': 6622, 'основной': 6561, 'информационный': 6452, 'решение': 5816, 'дисциплина': 5802, 'деятельность': 5534, 'программный': 4893, ...})

### Удаление слов
Удалим слова, которые встречаются менее 4 раз и слова, встречающиеся более 2000 раз, а также некоторые контекстные слова и опечатки.

In [13]:
context_stop = ['раздел', 'результат', 'освоение', 'применение', 'использование', 'изучение', 'практика',
    'представление', 'модуль', 'владеть', 'знать', 'уметь', 'применять', 'понятие', 'формирование',
    'создание', 'развитие', 'получать', 'иметь', 'опыт', 'студент', 'изучение', 'структура', 'умение',
    'использовать', 'общий', 'организация', 'опк', 'oпк', 'пк', 'ук', 'ок', 'принцип', 'назначение', 'цель',
    'способный', 'определение', 'построение', 'вид', 'проект', 'особенность', 'стр', 'стp', 'cтp', 'кoд',
    'б1', 'выпускник', 'иной', 'развивать', 'текущий', 'важный', 'ечас', 'позволять',  'направление',
    'бакалавр', 'лекция', 'находить', 'понимать', 'иметься', 'обладать', 'тема', 'ых', 'ый', 'подготовка',
    'самостоятельно', 'профиль', 'научать', 'специалист', 'иса', 'ита', 'осваивать', 'выпускной',
    'общепрофессиональный', 'a', 'разный', 'решать', 'июль', 'ока', 'oка', 'окa', 'квалификационный',
    'осуществление', 'первый', 'второй', 'третий', 'четвертый', 'пятый',  'комп', 'тенция', 'граммный',
    'отчет', 'весь', 'контрольный', 'овладение', 'самостоятельный', 'зачетный', 'единица',
    'давать', 'работать', 'выбирать', 'свой', 'курс', 'формировать','произ', 'водство', 'шение',
    'зад', 'ние', 'информ', 'ин', 'ровать', 'ного', 'ре', 'пoмещение', 'помещениe', 'наименование', 'также', 
]

words_most = []
words_least = []
most = 2000
least = 4
for w in d.keys():
    if d[w] >= most:
        words_most.append(w)
    elif d[w] <= least:
        words_least.append(w)

upd_wstop = set(words_most + words_least + typo_words + context_stop)
len(upd_wstop)

12499

In [14]:
fd_counter_trunc = FreqDist()
df_trunc = df.copy()

start = time.time()
df_trunc.text = df_trunc.text.apply(process_text, f_lemmatizer=None, wstop=upd_wstop, counter=fd_counter_trunc)
print('Time spent: {} s.'.format(time.time() - start))

Time spent: 3.319998025894165 s.


In [15]:
len(list(dict(fd_counter_trunc)))

6538

In [16]:
jsonStr_trunc = json.dumps(dict(fd_counter_trunc.most_common()), ensure_ascii=False)
file = open('JsonCounter_trunc.json', 'w')
file.write(jsonStr_trunc)
file.close()

Удалим дубликаты.

In [17]:
df_trunc.drop_duplicates(subset='text', inplace=True)
rewrite('files_lemm', df_trunc)
len(df_trunc)

2559

Посмотрим на данные.

In [18]:
print('Число университетов: {}'.format(len(df_trunc['univ'].unique())))
print('Число направлений: {}'.format(len(df_trunc['prog'].unique())))

Число университетов: 12
Число направлений: 54


In [19]:
df_trunc['univ'].unique()

array(['АГТУ', 'АГУ', 'АлтГТУ', 'АлтГУ', 'АмГУ', 'БашГУ', 'БГТУ',
       'БГУ Петровского', 'ВолгГТУ', 'ВолГУ', 'ВУиТ', 'ИМСИТ'],
      dtype=object)